In [ ]:
oboe_automl_path = '../../../oboe/automl/'
metafeature_folder_path = 'selected_OpenML_dataset_metafeatures'

In [ ]:
import numpy as np
import pandas as pd
import sys
import re
import os
from sklearn.model_selection import LeaveOneOut
from tqdm import tqdm
sys.path.append(oboe_automl_path)
import multiprocessing as mp

In [ ]:
import linalg
import convex_opt
from sklearn.model_selection import LeaveOneOut
import multiprocessing

In [ ]:
num_cores = multiprocessing.cpu_count()

In [ ]:
errmtx_df = pd.read_csv(os.path.join(oboe_automl_path, 'defaults/error_matrix.csv'), index_col=0, header=0)
errmtx = errmtx_df.values
runtime_df = pd.read_csv(os.path.join(oboe_automl_path, 'defaults/runtime_matrix.csv'), index_col=0, header=0)
runtime = runtime_df.values
dataset_sizes_df = pd.read_csv(os.path.join(oboe_automl_path, 'defaults/dataset_sizes.csv'), index_col=0, header=0)

In [ ]:
ind_errmtx = errmtx_df.index.astype(int)
ind_metafeatures = [int(re.findall("\\d+", file)[0]) for file in os.listdir(metafeature_folder_path) if file.endswith('.csv') and 'sizes' not in file]
ind_common = list(set(ind_errmtx).intersection(set(ind_metafeatures)))

In [ ]:
def indloc(indices, ind):
    return np.where(np.array(indices)==ind)[0][0]

In [ ]:
errmtx_common = errmtx_df.loc[ind_common].values
runtime_common_df = runtime_df.loc[ind_common]
runtime_common = runtime_common_df.values

In [ ]:
loo = LeaveOneOut()

In [ ]:
#standard PCA
X_pca, Y_pca, _ = linalg.pca(errmtx, threshold=0.01)
#show how many principal components correspond to eigenvalues larger than 1% of the largest
linalg.approx_rank(errmtx_common, threshold=0.01)

In [ ]:
errmtx_pred = np.zeros(errmtx_common.shape)

runtime_limit = 4
scalarization = 'D'
rank = linalg.approx_rank(errmtx_common, threshold=0.01)
errors_encountered = []
index_no_error = []

# leave-one-out cross-validation across datasets
for train_index, test_index in tqdm(loo.split(errmtx_common)):
    try:
        new_row = np.zeros((1, errmtx_common.shape[1]))
        t_predicted = convex_opt.predict_runtime(tuple(np.array(dataset_sizes_df.loc[ind_common[test_index[0]]])), runtime_matrix=runtime_common_df.iloc[train_index])
        valid = np.where(t_predicted <= num_cores * runtime_limit/2)[0]
        Y = Y_pca[:, valid]
        v_opt = convex_opt.solve(t_predicted[valid], runtime_limit/4, num_cores, Y, scalarization)
        to_sample = valid[np.where(v_opt > 0.9)[0]]
        new_row[:, to_sample] = errmtx_common[test_index, to_sample]
        errmtx_pred[test_index, :] = linalg.impute(errmtx_common, new_row, to_sample, rank=rank)
        index_no_error.append(test_index)
    except:
        print("Error encountered on dataset {}".format(ind_common[test_index[0]]))
        errors_encountered.append(ind_common[test_index[0]])        
    

index_no_error = [item[0] for item in index_no_error]

In [ ]:
# save results
errmtx_common_no_error = errmtx_common[index_no_error, :]
errmtx_pred_no_error = errmtx_pred[index_no_error, :]
percentage_of_error = np.linalg.norm(errmtx_pred_no_error - errmtx_common_no_error, axis=1) / np.linalg.norm(errmtx_common_no_error, axis=1)

In [ ]:
pd.DataFrame(ind_common, index=None, columns=None).to_csv("ind_oboe.csv", index=None, header=None)

In [ ]:
pd.DataFrame(errmtx_common_no_error, index=None, columns=None).to_csv("errmtx_common_no_error_oboe_{}_{}.csv".format(runtime_limit, scalarization), index=None, header=None)

pd.DataFrame(errmtx_pred_no_error, index=None, columns=None).to_csv("errmtx_pred_no_error_oboe_{}_{}.csv".format(runtime_limit, scalarization), index=None, header=None)

pd.DataFrame(percentage_of_error, index=None, columns=None).to_csv("percentage_of_error_oboe_{}_{}.csv".format(runtime_limit, scalarization), index=None, header=None)